<a href="https://colab.research.google.com/github/josedlhm/Trump_TextGenerator/blob/main/Trump_Speech_Generator_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install transformers

we install git lfs so we can upload our model to the hugging face repository. 

In [ ]:

!sudo apt-get install git-lfs

We prompt a login to the Huggingface website so that we can upload our model to our repository. This log in generates a token that is then used to upload the model. 


In [ ]:
!transformers-cli login

WARNING! `transformers-cli login` is deprecated and will be removed in v5. Please use `huggingface-cli login` instead.

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: josedlhm
Password: 
ERROR:root:HfApi.login: This method is deprecated in favor of `set_access_token`.
Login successful
Your token: NtPxHkQJnihsVroRznLXMbmjMpDmZBrJnrLqukFCDQLrURcBPBpuZNdesInPdSKMhmCwWCoGKaVZUsXHbjepTstGCYPKoIVsDSxPSspeaqRJQZnqeInoohHzuaZLoims 

Your to

Here we upload datasets to the environment. These can be either the tweets or the speeches from the 2016 campaign. 

Both  are in a json format, and in our github repository. 

We got the speeches here https://github.com/StoneCypher/trump-speeches

We got the tweets here https://www.kaggle.com/ayushggarg/all-trumps-twitter-insults-20152021

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving corpus_trump_speeches.json to corpus_trump_speeches.json


This chunk of code splits the dataset into training and testing parts, and assigns each part to a destination path. 

We build a function which takes the already split datasets that concatenates teh text into a single string and assigns them to a destination path. 
We call the function immediately after defining it.

Finally, we print the length of the training and testing datasets to see if the function worked. 

Note: when using the speeches data set, one should select the 'text' column within texts, while when using the tweets corpus, one should select the 'tweet' column.  

More specifically, we import the necessary libraries: re to use regular expressions, json to read json files and sklearn to split the dataset. 



In [ ]:
import re 
import json  
from sklearn.model_selection import train_test_split 


with open('corpus_trump_speeches.json') as f:
    data = json.load(f)

train, test = train_test_split(data,test_size=0.15) 


def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = '' 
    for texts in data_json: 
        summary = str(texts['text']).strip() 
        summary = re.sub(r"\s", " ", summary) 
        data += summary + " " 
    f.write(data) 

build_text_files(train,'train_dataset.txt') 
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train))) 
print("Test dataset length: "+ str(len(test)))

Train dataset length: 63677
Test dataset length: 11238


Here we import the tokenizer using AutoTokenizer from the transformers library. We specify the openai-gpt tokenizer, and assign to our local tokenizer object. 

We also specify the training and testing paths. 


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-gpt")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/448k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Here we define a function that tokenizes our two datasets. 

In the 'load_dataset' function, it takes two paths and a tokenizer as arguments, we tokenize the text of each dataset and assign to the appropriate file path, which we defined above. We also create a datacollator using openai-gpt's tokenizer. 

The function returns the tokenized training and testing data sets, as well as the data collator. 

We call the function immediately after. 


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset( 
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128) 
     
    test_dataset = TextDataset( 
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(                                         
        tokenizer = tokenizer, mlm=False 
    )
    return train_dataset, test_dataset, data_collator 

train_dataset, test_dataset, data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Token indices sequence length is longer than the specified maximum sequence length for this model (843928 > 512). Running this sequence through the model will result in indexing errors


We install the dataset library from huggingface, which has several metric options. 


In [ ]:
!pip install datasets

We specify accuracy as the metric to test our mode. Accuracy  is the proportion of correct guesses. 
While accuracy might be more appropriate for another kind of model,  we were not sure which metric to use. 
We considered using bleurt (apparently better for text generation), but it recommends using a larger dataset 

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Here we simply specify our model, which is openai-gpt with LMHead, (which the version required for text generation)

Then, we specify the training arguments. Here we select an output directory on which to save the model. This is a local repository in the colab environment, we don't upload the model to hugging face til later on.  We also specify features of training: the number of epochs, batch size, evaluation steps, etc. 

We also define our trainer. Here we specify several objects that we have already defined: our model, the training arguments, the data collator, the training data, the testing data, the metrics and data collator. 

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("openai-gpt") 
                                            

training_args = TrainingArguments(
    output_dir="josedlhm/trump-speeches", 
    overwrite_output_dir=True, 
    num_train_epochs=2, 
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=8,  
    eval_steps = 400, 
    save_steps=800, 
    warmup_steps=500,
    prediction_loss_only=True,
    evaluation_strategy="epoch" 
    )

trainer = Trainer(
    model=model, 
    args=training_args, 
    data_collator=data_collator,  
    train_dataset=train_dataset, 
    eval_dataset=test_dataset,  
    compute_metrics = compute_metrics, 
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:698: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/457M [00:00<?, ?B/s]

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Here we train our model

In [ ]:
trainer.train() 

***** Running training *****
  Num examples = 6593
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3298


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

Here we save our model to local repository 


In [ ]:
trainer.save_model()  

Saving model checkpoint to josedlhm/trump-speeches
Configuration saved in josedlhm/trump-speeches/config.json
Model weights saved in josedlhm/trump-speeches/pytorch_model.bin


We added this bit to this so that colab wouldn't shut down during the 8 hours of training. 

In [ ]:
while True:pass 

Here we push our model to the github repository. We pushed the speeches model to the repository called "josedlhm/new_model" and the model trained on tweets to the repository called "josedlhm/trump_speeches" 

In [ ]:
model.push_to_hub("josedlhm/new_model")  


Configuration saved in josedlhm/new_model/config.json
Model weights saved in josedlhm/new_model/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.39k/457M [00:00<?, ?B/s]

To https://huggingface.co/josedlhm/new_model
   f7e2f9c..ba7dc23  main -> main



'https://huggingface.co/josedlhm/new_model/commit/ba7dc235947cb822ae0599b8b9e4f0ac0a917f5f'